In [ ]:
! pip install langchain
! pip install ctransformers
! pip install sentence-transformers
! pip install faiss-cpu

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from lanchain.llms import Ctransformers
from langchain.chains import CoversationBufferMemory
from langchain.chains import ConversionalRetrivalChain
import sys


In [ ]:
DB_FAISS_PATH='vectorstore/db_faiss'
loader=CSVLoader(file_path="data/2019.csv",encoding='utf-8',csv_args={'delimiter':','})
data=loader.load()

In [ ]:
print(data)

In [ ]:
# split the text into chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)


In [ ]:
text_chunks=text_splitter.split_documents(data)


In [ ]:
print(len(text_chunks))

In [ ]:
# download sentence transformers import Embedding from the hugging face

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
# converting the text chunks into embeddings  and savings the embeddings into FAISS Knowledge Base

In [ ]:
docsearch =FAISS.from_document(text_chunks,embeddings)


In [ ]:
# query = "What is the value of GDP per capita of Finland provided in the data?"


In [ ]:
docs=docsearch.similarity_search(query,k=3)


In [ ]:
print("Results",docs)

In [ ]:
llm=Ctransformers(model="models/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type'llama',
                  max_new_tokens=512,
                  temperature=0.1)

In [ ]:
qa=ConversationalRetrivalChain.from_llm(llm,retriver=docsearch.as_retriver())


In [ ]:
while True:
  chat_history =[]
  query=input(f'Input Prompt:')
  if query == exit:
    print('Exiting')
    sys.exit()
  if query == '':
    continue
  result = qa({"question":query,"chat_history":chat_history})
  print("Response:",result['answer'])
